In [2]:
import pandas as pd

In [8]:
info = pd.read_excel('./data/raw/2009Q1.xlsx', skiprows = 14)
info

,Rank,Insurer Name,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Policies In Force,Unnamed: 9,...,Claims with Arbitration,Unnamed: 56,Claims with Appraisal,Unnamed: 58,Claims with Sink Hole Eval,Unnamed: 60,Claims with Settlement Conf,Unnamed: 62,Claims with Other,Unnamed: 64
0,1,CITIZENS PROPERTY INSURANCE CORPORATION,NaN,NaN,NaN,NaN,NaN,NaN,1005077,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
1,2,STATE FARM FLORIDA INSURANCE COMPANY,NaN,NaN,NaN,NaN,NaN,NaN,853867,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
2,3,UNIVERSAL PROPERTY & CASUALTY INSURANCE COMPANY,NaN,NaN,NaN,NaN,NaN,NaN,498139,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
3,4,"ST. JOHNS INSURANCE COMPANY, INC.",NaN,NaN,NaN,NaN,NaN,NaN,204409,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
4,5,UNITED SERVICES AUTOMOBILE ASSOCIATION,NaN,NaN,NaN,NaN,NaN,NaN,163850,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,210,AMERICAN GENERAL PROPERTY INS CO OF FLORIDA,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
210,211,EMC PROPERTY & CASUALTY COMPANY,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
211,212,STATE NATIONAL INSURANCE COMPANY INC.,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
212,213,"TESLA PROPERTY & CASUALTY, INC.",NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,...,NaN,0%,NaN,0%,NaN,0%,NaN,0%,NaN,0%
